In [1]:
#%% General Imports
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt

#sklearn imports
import sklearn.linear_model as linear_model

#pyomo
from pyomo.environ import *


In [4]:
REPO = 'viscosity_liquid_transfer_Pablo'
parent_path = os.getcwd().split(REPO)[0]

# 2. IPOPT_LiqTransfer class

In [14]:
class IPOPT_LiqTransfer:

    def __init__(self, liquid_name):
        self.liquid_name = liquid_name
        self._data = None
        self.features = ['aspiration_rate','dispense_rate']
        self.objectives = ['%error']
        self.bmax = 1.25
        self.bmin = 0.1
        self._latest_suggestion = None
        self._latest_volume = None
    
    def set_data(self,df):
        df['time_asp_1000'] = 1000/df['aspiration_rate'] + 1000/df['dispense_rate'] + df['delay_aspirate'] + df['delay_dispense']
        self._data = df

    
    def data_from_csv(self,file_name):
        data = pd.read_csv(file_name)
        data = data.loc[:,['liquid','pipette','volume','aspiration_rate','dispense_rate','blow_out_rate','delay_aspirate','delay_dispense','delay_blow_out','%error']]
        self.set_data(data)

    def update_data(self,error):
        updated_data = pd.concat([self._data,self._data.iloc[[-1]]],ignore_index=True)
        updated_data.loc[updated_data.last_valid_index(),'volume'] = self._latest_volume
        updated_data.loc[updated_data.last_valid_index(),'aspiration_rate']  = self._latest_suggestion['aspiration_rate'][0]
        updated_data.loc[updated_data.last_valid_index(),'dispense_rate']  = self._latest_suggestion['dispense_rate'][0]
        updated_data.loc[updated_data.last_valid_index(),'%error'] = error
        self.set_data(updated_data)
        return self._data
                                
    def xy_split(self,volume=1000):
        df_train = self._data.where(self._data['volume']==volume).dropna(how='all').copy()
        x_train = df_train[self.features]
        y_train = df_train[self.objectives]
        return x_train,y_train

    def set_bounds(self, x_train):
        return x_train.iloc[0,0]*self.bmin, x_train.iloc[0,0]*self.bmax

    def fit_lin(self,volume=1000):
        lin_model = linear_model.LinearRegression()
        x_train,y_train = self.xy_split(volume=volume)
        
        min,max = self.set_bounds(x_train)

        lin_model.fit(x_train,y_train)


        m1,m2 = lin_model.coef_.tolist()[0]
        b= lin_model.intercept_.tolist()[0]

        return m1,m2,b,min,max
    

    
    

    
    def optimized_suggestions(self,volume= 1000):
        self._latest_volume = volume        
        
        model = ConcreteModel()
        m1,m2,b,min,max = self.fit_lin(volume=volume)

        def obj_time_for_1000(m):
    
            return 1000/m.x1 + 1000/m.x2 + 10
        # # Define decision variables

        model.x1 = Var(initialize= (min+max)/2, bounds = (min,max))
        model.x2 = Var(initialize= (min+max)/2, bounds = (min,max))

        model.obj = Objective(rule= obj_time_for_1000, sense=minimize)
        
        # Define constraints
        model.constraints = ConstraintList()
        model.constraints.add(expr= m1 * model.x1 + m2 * model.x2 + b <= 2.5)
        model.constraints.add(expr= m1 * model.x1 + m2 * model.x2 + b >= -2.5)
        model.constraints.add(expr= model.obj >= 10)

        solver = SolverFactory('ipopt')
        solver.solve(model)
        
        self._latest_suggestion = pd.DataFrame({'aspiration_rate':model.x1.value,'dispense_rate':model.x2.value}, index=[0])
        
        return self._latest_suggestion
        

        

In [15]:
# Change according to experiment
liquid_name = 'Viscosity_std_1275' 

# Do not change
liq = IPOPT_LiqTransfer(liquid_name)
liq.data_from_csv(parent_path+REPO+'\\Opentrons_experiments\\BOTorch_optimization\\CCF_initialization\\BOTorch_optimization_CCF_' +liquid_name+'.csv')
liq._data

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.657700
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.657700
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.657700
3,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.526160
4,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.526160
5,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.526160
6,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578
7,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578
8,Viscosity_std_1275,p1000,300,27.576257,2.206101,0,5,5,0,5.883700,499.551578
9,Viscosity_std_1275,p1000,1000,2.206101,27.576257,0,5,5,0,-9.043040,499.551578


In [22]:
liq.optimized_suggestions(volume=500)

,aspiration_rate,dispense_rate
0,9.677176,8.03751


In [23]:
liq.update_data(-0.840723)

,liquid,pipette,volume,aspiration_rate,dispense_rate,blow_out_rate,delay_aspirate,delay_dispense,delay_blow_out,%error,time_asp_1000
0,Viscosity_std_1275,p1000,1000,22.061005,22.061005,0,5,5,0,-15.934066,100.657700
1,Viscosity_std_1275,p1000,500,22.061005,22.061005,0,5,5,0,-29.212454,100.657700
2,Viscosity_std_1275,p1000,300,22.061005,22.061005,0,5,5,0,-37.080281,100.657700
3,Viscosity_std_1275,p1000,1000,27.576257,27.576257,0,5,5,0,-24.015568,82.526160
4,Viscosity_std_1275,p1000,500,27.576257,27.576257,0,5,5,0,-31.524725,82.526160
5,Viscosity_std_1275,p1000,300,27.576257,27.576257,0,5,5,0,-34.790904,82.526160
6,Viscosity_std_1275,p1000,1000,27.576257,2.206101,0,5,5,0,-4.521520,499.551578
7,Viscosity_std_1275,p1000,500,27.576257,2.206101,0,5,5,0,-3.296703,499.551578
8,Viscosity_std_1275,p1000,300,27.576257,2.206101,0,5,5,0,5.883700,499.551578
9,Viscosity_std_1275,p1000,1000,2.206101,27.576257,0,5,5,0,-9.043040,499.551578
